In [1]:
import os, time
import torch
import torch_geometric
from datasets.BatchWSI import BatchWSI
from models.model_graph_mil import *
device = torch.device('cuda:0')

dataroot = '/home/weber50432/UCL/final_dissertation/test/graph_files/'
large_graph_pt = 'TCGA-BH-A0C1-01Z-00-DX1.21FE357E-B182-4397-BFEF-7E96E994236A.pt' # example input

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

/home/weber50432/miniconda3/envs/ML/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Graph Data Structure
- `N`: number of patches
- `M`: number of edges
- `centroid`: [N x 2] matrix containing centroids for each patch
- `edge_index`: [2 x M] matrix containing edges between patches (connected via adjacent spatial coordinates)
- `edge_latent`: [2 x M] matric containing edges between patches (connected via latent space)
- `x`: [N x 1024] matrix which uses 1024-dim extracted ResNet features for each iamge patch (features saved for simplicity)

In [2]:
data = torch.load(os.path.join(dataroot, large_graph_pt))
data

Data(x=[42430, 1024], edge_index=[2, 339440], edge_latent=[2, 339440], centroid=[42430, 2])

In [5]:
print(data.edge_latent[:, :10])
print(data.edge_index[:, :10])  # 顯示前10條 edge

tensor([[    0,     0,     0,     0,     0,     0,     0,     0,     1,     1],
        [27479, 29109, 31371, 31605, 36505, 38065, 10195, 13223, 27479, 29109]])
tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  1,  1],
        [ 1,  2,  5,  3,  6, 17, 18,  7,  0,  2]])


In PyTorch Geometric, inference on large graphs is very tractable. Here, adjacency matrices are stacked in a diagonal fashion (creating a giant graph that holds multiple isolated subgraphs), and node and target features are simply concatenated in the node dimension

This procedure has some crucial advantages over other batching procedures:

- GNN operators that rely on a message passing scheme do not need to be modified since messages still cannot be exchanged between two nodes that belong to different graphs.

- There is no computational or memory overhead. For example, this batching procedure works completely without any padding of node or edge features. Note that there is no additional memory overhead for adjacency matrices since they are saved in a sparse fashion holding only non-zero entries, i.e., the edges. 
- For more details, see the advanced mini-batching FAQ in: https://pytorch-geometric.readthedocs.io/en/latest/notes/batching.html

In [3]:
data = BatchWSI.from_data_list([torch.load(os.path.join(dataroot, large_graph_pt)), 
                                torch.load(os.path.join(dataroot, large_graph_pt))])
data

TypeError: 'method' object is not iterable

### Inference + Backprop using 23K patches

In [ ]:
data = BatchWSI.from_data_list([torch.load(os.path.join(dataroot, large_graph_pt))])
data = data.to(device)
data

In [ ]:
model_dict = {'num_layers': 4, 'edge_agg': 'spatial', 'resample': 0, 'n_classes': 1}
model = PatchGCN_Surv(**model_dict).to(device)
print("Number of Parameters:", count_parameters(model))

### Example Forward Paas + Gradient Backprop
start = time.time()
out = model(x_path=data)
out[0].backward()
print('Time Elapsed: %0.5f seconds' % (time.time() - start))

### Inference + Backprop using 92K patches

In [ ]:
### Simulating a very large graph (containing 4 subgraphs of 23K patches each)
data = BatchWSI.from_data_list([torch.load(os.path.join(dataroot, large_graph_pt)), 
                                torch.load(os.path.join(dataroot, large_graph_pt)),
                                torch.load(os.path.join(dataroot, large_graph_pt)),
                                torch.load(os.path.join(dataroot, large_graph_pt))])
data = data.to(device)
data

In [ ]:
model_dict = {'num_layers': 4, 'edge_agg': 'spatial', 'resample': 0, 'n_classes': 1}
model = PatchGCN_Surv(**model_dict).to(device)
print("Number of Parameters:", count_parameters(model))

### Example Forward Paas + Gradient Backprop
start = time.time()
out = model(x_path=data)
out[0].backward()
print('Time Elapsed: %0.5f seconds' % (time.time() - start))

Assuming worst case scenario that every graph has ~100K patches, for a dataset of 1000 WSIs, an epoch would take 3.43 minutes, with 20 epochs taking ~ 1 hour.